In [ ]:
import pandas as pd
import numpy as np
# Write a function to make this easier and help w/ data pipelines
#read in the data, understand what is included, and find missing values if any

df = pd.read_csv('marketing_data.csv', skiprows=1) #dropped first row b/c it was not the column names
def remove_outliers(df, col_list, whisk=1.5):
    for column in col_list:
        q1, q3= np.percentile(df[column], [25, 75])
        iqr = q3-q1 #sets the main interval 
        lower_limit = q1 - whisk*iqr #whisk is more flexible than 1.5, reco to use this to maintain flexibility
        upper_limit = q3+ whisk*iqr
        n_r_before = df.shape[0]
        df= df[(df[column]>=lower_limit) & (df[column]<= upper_limit)] #replace original df with a new df
        n_after = df.shape[0]
        print(column, "Before:", n_r_before, "After:", n_after)
        return(df)


def clean_currency(x):
    return(str(x).replace('$', '').replace(',', ''))
df.columns=df.columns.str.strip()


df.dtypes
cols = list(df)
cols
df.info()
df.describe()
na_missing_values = df.isna().sum()
na_missing_values
missing_values = df.isnull().sum()
missing_values
cols = list(df.select_dtypes('object').columns)  #to check what else is an object - we need to code columns as a string.
cols 
df.head()

In [ ]:
df.isnull().sum() #check for missing values

In [ ]:
df['Education'] = df['Education'].astype(pd.StringDtype())
df['Marital_Status'] = df['Marital_Status'].astype(pd.StringDtype())
df['Income'] = df['Income'].astype(pd.StringDtype()) #note that col header for "income" has spaces before and after the header title. This impacts analysis, so be careful!
df['date'] = pd.to_datetime(df['Dt_Customer'], format='mixed')  #create a new variable for easier processing
df['Country'] = df['Country'].astype(pd.StringDtype())
df.info()  #to check if our transform works (it does!)

In [ ]:
#addres missing values. We do this using the mean 
#df['Income'] = df['Income'].fillna(df['Income'].mean())
df['Income']= df['Income'].astype(pd.StringDtype())
df['Income'].info()

In [ ]:
def clean_currency(x):
    return(str(x).replace('$', '').replace(',', ''))
df.columns=df.columns.str.strip()
df['Income']=df["Income"].apply(clean_currency)
df['Income'] = pd.to_numeric(df['Income'], errors='coerce')
df['Income'] = df['Income'].fillna(df['Income'].mean())
df['Income'].info()

Exploring columns and encoding variables

In [ ]:
marriage_count = df['Marital_Status'].value_counts() #to understand spread of data
marriage_count

In [ ]:
replacements = [('Married',  '1') , ('Together', '1'),    ('Single', '0'),('Divorced', '0'),('Widow' , '0'),('Alone', '0'),('YOLO' ,'0'),('Absurd', '0')]
for old, new in replacements: 
    df['Marital_Status'].replace(old, new, inplace=True)
df['Married']= pd.to_numeric(df['Marital_Status'], errors='ignore') #received a note that this method is being deprecated; used an alternative for encoding education below

In [ ]:
kid_count = df['Kidhome'].value_counts()
kid_count


In [ ]:
teen_count = df['Teenhome'].value_counts()
teen_count

Kids and teens:
- We can do one-hot encoding for families with kids and teens but it may be more beneficial to roll kids and teens together so we know how many additional family members are in the house.

**CALLOUT IN THIS APPROACH:** we are assuming that we just need to know how many others in the household aside from the purchaser, not the age.

In [ ]:
#merge the different columns to reflect whether a house has kids or not (regardless of age) 
x= df['Kidhome']
y=df['Teenhome']
Total_kids = pd.concat([x,y], axis=1)
Total_kids = x+ y
df['children_home'] = Total_kids
df.describe()

In [ ]:
education_count = df['Education'].value_counts()
education_count

*Education*
- It would be good to encode education.
    - We can't identify all the education statuses cleanly because of the variation in responses, but we can reliably identify higher education via PhD and Masters categories. 
    - There is an order to the level of education, so manual encoding is the best option.

In [ ]:
#coded education values 
edu_map = {'Graduation': '1', 'PhD': '1', 'Master': '1', '2n Cycle': '0', 'Basic': '0'}
x=df.assign(Higher_ed  = df['Education'].map(edu_map)) #this mapping is a little cleaner than the above approach. Will use this from now often!
df['Higher_ed'] = x['Higher_ed']

In [ ]:
#encoded country values
country_spread = df['Country'].value_counts()
country_spread # we should encode these too for easier processing later on

country_map = {'SP': '1', 'SA':'2', 'CA':'3', 'AUS':'4', 'IND': '5', 'GER':'6', 'US':'7', 'ME':'8'}
y = df.assign(Country_coded = df['Country'].map(country_map))
df['Country'] = y['Country_coded']
df.head()

Looking at the spread across countries, we see that our data is very concentrated in Spain. This is something we should consider as we move through data processing, as this could introduce a bias in interpretation. We also needed to code the country locales so we can process the data easier later on.


**3. Create variables (a.k.a, feature engineering)**

We need to do some work to make our fields suitable for analysis. 
We've already coded marriage status, kids at home, and the country location; these are two new columns we've added to the dataset. 


Now, we must create an age field:

In [ ]:
#create an age field calculating from year born
import datetime
year_born = 2024 - df['Year_Birth']
df['Age']= year_born
df.head()

Create a variable for "average spend per purchase" aka avg_spend

In [ ]:
df['Purchases'] = df['NumStorePurchases'] + df['NumDealsPurchases'] + df['NumWebPurchases'] + df['NumCatalogPurchases']

df['Cost']  = (df['MntMeatProducts'] + df['MntWines'] + df['MntSweetProducts'] + df['MntFishProducts'] + df['MntFruits'] + df['MntGoldProds'])

average_spend = df['Purchases']/df['Cost']
df['average_spend'] = average_spend
df.describe()

We can see that on average, people made around 15 purchases over the past 2 years, spending around $600 on average per purchase.


Now let's group some columns together to make it easier to work with!

In [ ]:
people = df[['Higher_ed','Married','Income','children_home','Age','Country', 'average_spend','Cost','Purchases']]

In [ ]:
promotion_activity = df[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']]


In [ ]:
#purchases by vertical (place)
purchases = df[['NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']]
purchases.info()

In [ ]:
#product revenue distribution
product = df[['MntFishProducts', 'MntFruits', 'MntMeatProducts', 'MntWines', 'MntSweetProducts', 'MntGoldProds', 'Cost']]
product.describe()

# Visualizations 

Boxplots and histograms to see distribution and outliers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
people_hist = people.hist(column =['Higher_ed','Income','Age','Country', 'average_spend'], bins = 30)
plt.tight_layout()
plt.show()
people_box= people.boxplot(column =['Income'],vert=False, showfliers=True)  #spread of income values
plt.show()
people_box_age = people.boxplot(column=['Age'], vert=False, showfliers=True)
plt.show()

We can see that some age values are outliers; there are 3. 

In [ ]:
purchases_hist = purchases.hist(column=['NumCatalogPurchases', 'NumWebPurchases', 'NumDealsPurchases','NumStorePurchases'],bins = 20)
plt.tight_layout()
plt.show()
purchases_boxplot= purchases.boxplot(vert=False, showfliers=True) 
plt.show()

Purchase channels are right-skewed and there are a lot of outliers in the purchases made on deal; this could indicate that there were some attractive deals at some point, which drove more purchases.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt, style
import seaborn as sns

In [ ]:
#spread of product revenue
product_hist = product.hist(column = ['MntFishProducts', 'MntFruits', 'MntGoldProds', 'MntSweetProducts', 'MntWines', 'MntMeatProducts'], bins=30)
plt.tight_layout() #adjusts spacing, etc.
plt.show() #clean up display

product_box = product.boxplot(vert = False, showfliers=False) #removed outliers to get a better sense of the data spread. It looks like wine is the most purchased product, followed by meat products.
plt.show()

I included cost here to highlight the distribution of revenue across products compared to total expenditure.

In [ ]:
#promotion activity
import numpy as np
import matplotlib.pyplot as plt
z = promotion_activity.sum()
z
promo_plot =z.plot(column= ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response'], rot=45)
plt.title('Promotion Campaign Acceptance Rates')
plt.xlabel('Campaigns')
plt.ylabel('Promotion Engagement')
plt.tight_layout() #adjusts spacing, etc.
plt.show() 

Looking at this, we can see that campaign 2 is the worst-performing campaign while the latest campaign, Response, is the most successful of the efforts. Note that a box plot won't show us too much information, as we're concerned about overall campaign performance, so rendering a boxplot wouldn't make sense.

# Correlation and Significance Testing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#correlation between amount spent on types of products 

plt.figure(figsize=(10,6))
sns.heatmap(product.corr(), annot=True, cmap='viridis')
plt.title('Product Vertical Correlation', fontsize=16)
plt.show()

People spend the most on wine and meat products. These behave similarly in terms of money spent on a purchase (e.g., cost), as indicated by the strong, positive relationship shown in the corerlation matrix.

In [ ]:
import scipy.stats
from scipy.stats import pearsonr
age_spend_corr = pearsonr(df['Age'], df['Cost'])
age_spend_corr 

Pvalue is nearly 0, so we reject H0 that there is no relationship between age and how much is spent. As we see age increase, we should also see an increase in amount spent. 

Sales channel cannibalization

We can look for this to see if variables are linked; if they are, then that means we can probably remove one channel to concentrate business. We can look into this using a correlation test.

In [ ]:
plt.figure(figsize=(10,6)) #to see relationships across channels
sns.heatmap(purchases.corr(), annot=True, cmap='viridis')
plt.title('Sales Channel Correlation', fontsize=16)
plt.show()

In [ ]:
store_cat_correl = pearsonr(df['NumStorePurchases'], df['NumCatalogPurchases'])
store_cat_correl #specific investigation between store and catalog purchases

We can see that purchases made in the catalog are positively linked to store purchases, but this is a moderately strong relationship (r=0.518). 

- H0 = no relationship between store purchases and catalog purchases
- H1 = relationship between store purchases and catalog purchases 

Correlation of store and catalog purchases = 0.52, pvalue approx 0. This means we can confidently reject the null hypothesis and say that there is a moderate relationship between shopping in the store versus the catalog. 

In [ ]:
# to find out if homes with kids purchase on line more (spoiler: they don't)
purchases_with_kids =df[['NumCatalogPurchases', 'NumWebPurchases', 'NumDealsPurchases','NumStorePurchases', 'children_home']]
purchases_with_kids.describe()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(purchases_with_kids.corr(), annot=True, cmap='viridis')
plt.title('Kids and purchase behaviors', fontsize=16)
plt.show()

In [ ]:
kids_purchase_corr = pearsonr(purchases_with_kids['children_home'], purchases_with_kids['NumWebPurchases'])
kids_purchase_corr

In [ ]:
kid_web_correl= np.corrcoef(purchases_with_kids['children_home'], purchases_with_kids['NumWebPurchases'])
kid_web_correl #folks with children do not tend to shop more online

In [ ]:
scipy.stats.zscore(purchases_with_kids, axis = 0) #another way to confirm this

In [ ]:
product.head()

In [ ]:
product_units = pd.crosstab(index=df['Country'], columns=df['Cost'].sum())

In [ ]:
t = product_units.plot.bar()
#country map: 'SP': '1', 'SA':'2', 'CA':'3', 'AUS':'4', 'IND': '5', 'GER':'6', 'US':'7', 'ME':'8'}
plt.xticks((0,1, 2, 3, 4, 5, 6, 7), ('SP', 'SA', 'CA', 'AUS', 'IND', 'GER', 'US', 'ME')) #set index to 0, as this is how python operates. This plot is in line with our table value above!
plt.title('Products purchased by Country', fontsize = 16)
plt.show() 

In [ ]:
#trake out cost from bar graph
z = product.sum()
prod_plot = z.plot.bar( rot=45)
plt.title('Product spend by vertical', fontsize = 16)
plt.show()

In [ ]:
df['Response'].sum() #check to see how many responded to most recent campaign

In [ ]:
data = df[['children_home', 'Cost']]

In [ ]:
x = data['children_home']
y = data['Cost']

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.bar(x,y)
plt.xlabel('Number of children', fontweight ='bold', fontsize = 15)
plt.ylabel('Cost', fontweight ='bold', fontsize = 15)
plt.xticks(np.arange(min(x), max(x)+1, 1.0))
plt.show()
data.sort_values('children_home', ascending=True)

In [ ]:
import scipy.stats
from scipy.stats import chi2_contingency
data = df[['Country','Purchases']]
data = pd.DataFrame(data)
stat, p, dof, expected = chi2_contingency(data)
country_vol = stat, p, dof, expected
country_vol

There is a significant relationship between the country and how many purchases are made.

In [ ]:
df['Country'] = df['Country'].astype(int)
country_purchase_corr = np.corrcoef(df['Country'], df['Purchases'])
country_purchase_corr

In [ ]:
import scipy.stats
from scipy.stats import chi2_contingency
data = df[['Country', 'Purchases']]
data = pd.DataFrame(data)
stat, p, dof, expected = chi2_contingency(data)
country_vol = stat, p, dof, expected
country_vol

In [ ]:
from scipy.stats import ttest_ind
stats.ttest_ind(data['Country'], data['Purchases'], equal_var=False)

In [ ]:
correlation_coefficient, p_value = scipy.stats.pearsonr(df['Country'], df['Purchases'])
# Interpretation of the correlation
if p_value < 0.05:
    print("The correlation is statistically significant.")
else:
    print("The correlation is not statistically significant.")
    
if correlation_coefficient > 0:
    print("There is a positive correlation between Country and purchases.")
elif correlation_coefficient < 0:
    print("There is a negative correlation between Country and purchases.")
else:
    print("There is no linear correlation between Country and purchases.")

In [ ]:
import numpy as np
sample_coprrel = np.corrcoef(df['Age'], df['Response'])

In [ ]:
#Calculate Pearson correlation coefficient and p-value
correlation_coefficient, p_value = scipy.stats.pearsonr(df['Age'], df['Response'])
# Interpretation of the correlation
if p_value < 0.05:
    print("The correlation is statistically significant.")
else:
    print("The correlation is not statistically significant.")
    
if correlation_coefficient > 0:
    print("There is a positive correlation between Age and most recent campaign acceptance.")
elif correlation_coefficient < 0:
    print("There is a negative correlation between Age and most recent campaign acceptance.")
else:
    print("There is no linear correlation between Age and most recent campaign acceptance.")